In [1]:
#Muhammad Hasham Ul Haq
#i200752@nu.edu.pk
%pip install transformers
%pip install accelerate
%pip install annoy


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/647.5 kB ? eta -:--:--
      --------------------------------------- 10.2/647.5 kB ? eta -:--:--
     -- ---------------------------------- 41.0/647.5 kB 495.5 kB/s eta 0:00:02
     ------- ------------------------------ 133.1/647.5 kB 1.1 MB/s eta 0:00:01
     ---------------- --------------------- 276.5/647.5 kB 1.7 MB/s eta 0:00:01
     ----------------------- -------------- 399.4/647.5 kB 1.9 MB/s eta 0:00:01
     -------------------------------- ----- 553.0/647.5 kB 2.2 MB/s eta 0:00:01
     -------------------------------------  645.1/647.5 kB 2.3 MB/s eta 0:00:01
     -------------------------------------- 647.5/647.5 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for annoy
Failed to build annoy
No

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [14 lines of output]
      C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\setuptools\installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
        warnings.warn(
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\annoy
      copying annoy\__init__.py -> build\lib.win-amd64-cpython-311\annoy
      copying annoy\__init__.pyi -> build\lib.win-amd64-cpython-311\annoy
      copying annoy\py.typed -> build\lib.win-amd64-cpython-311\annoy
      running build_ext
      building 'annoy.annoylib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('urdu_stories.csv')

# Create a new column for split ID (you can adjust this based on your requirements)
df['Split_ID'] = range(len(df))

# Drop the 'Title' column
df = df[['Urdu Story', 'Split_ID']]  # Add 'Split_ID' to the relevant columns

# Split the dataset into training and validation sets
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

# Save the preprocessed datasets
train_df.to_csv('train_dataset.csv', index=False)
valid_df.to_csv('valid_dataset.csv', index=False)

# Display a few rows of the training dataset
print("Training Dataset:")
print(train_df.head())

# Display a few rows of the validation dataset
print("\nValidation Dataset:")
print(valid_df.head())


Training Dataset:
                                            Urdu Story  Split_ID
228  میں نے کل بھی کہا تھا اور آج پھر کہہ رہا ہوں ک...       228
208  اف کیا بتاؤں تمہیں گھر کی شفٹنگ کتنا مشکل کام ...       208
96   میرے ابّو ریلوے میں ملازم تھے۔ اچانک زندگی کی ...        96
167  جب میری شادی منیر سے ہوئی ، ان دنوں وہ ایک سرک...       167
84   کوئی چوتھی بار آ کر فاطمہ نے کھڑکی سے باہر جھا...        84

Validation Dataset:
                                            Urdu Story  Split_ID
24   ہم کھاتے پیتے لوگ تھے۔ مجھے ملازمت کی ضرورت نہ...        24
6    چالیس دن کی تھی جب میری زندگی کی عجب کہانی کی ...         6
153  دلہن رخصت ہو کر بخیر و خوبی کار تک رسائی حاصل ...       153
211  موسم سرما نے پاؤں پاؤں چلنا شروع کیا تھا۔ سورج...       211
198  میرے دو چھوٹے بھائی تھے لیکن والدین ان سے زیاد...       198


In [3]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, DistilBertModel
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from torch import optim
from tqdm import tqdm
import torch
import pandas as pd
import numpy as np

# Tokenize the datasets
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

class UrduDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        # Preprocess label to handle non-integer values
        try:
            label = int(self.labels[idx])
        except ValueError:
            label = 0  # Set a default label or handle the case based on your specific requirements

        encoding = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}


train_dataset = UrduDataset(train_df['Urdu Story'].tolist(), train_df['Urdu Story'].tolist(), tokenizer)
valid_dataset = UrduDataset(valid_df['Urdu Story'].tolist(), valid_df['Urdu Story'].tolist(), tokenizer)

# Load model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Define training arguments
epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

# Training loop
for epoch in range(epochs):
    model.train()
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False):
        input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the fine-tuned model
model.save_pretrained("./distilbert-finetuned")
tokenizer.save_pretrained("./distilbert-finetuned")



C:\Users\hasha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('./distilbert-finetuned\\tokenizer_config.json',
 './distilbert-finetuned\\special_tokens_map.json',
 './distilbert-finetuned\\vocab.txt',
 './distilbert-finetuned\\added_tokens.json')

In [8]:
total_splits = 10
mini_lm_epochs = 3  # Set the number of training epochs
mini_lm_models = []  # Initialize the list to store mini-LM models

for split_id in range(total_splits):
    # Create mini-LM model
    mini_lm_model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(device)
    mini_lm_linear = torch.nn.Linear(mini_lm_model.config.hidden_size, tokenizer.vocab_size).to(device)
    mini_lm_optimizer = optim.AdamW(list(mini_lm_model.parameters()) + list(mini_lm_linear.parameters()), lr=5e-5)

    # Create mini-LM dataset for the current split
    mini_lm_dataset = UrduDataset(train_df[train_df['Split_ID'] == split_id]['Urdu Story'].tolist(),
                                  train_df[train_df['Split_ID'] == split_id]['Urdu Story'].tolist(), tokenizer)

    print(f"Dataset length for split {split_id}: {len(mini_lm_dataset)}")  # Add this line to check the length of the dataset

    # Skip training for empty datasets
    if len(mini_lm_dataset) == 0:
        print(f"Skipping training for split {split_id} as the dataset is empty.")
        continue

    # Mini-LM Training
    for epoch in range(mini_lm_epochs):
        mini_lm_model.train()
        mini_lm_loader = DataLoader(mini_lm_dataset, batch_size=8, shuffle=True)
        for batch in tqdm(mini_lm_loader, desc=f'Mini-LM Epoch {epoch + 1}/{mini_lm_epochs}', leave=False):
            input_ids, attention_mask, _ = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)
            mini_lm_optimizer.zero_grad()

            # Forward pass through mini-LM model
            outputs = mini_lm_model(input_ids, attention_mask=attention_mask)

            # Extract last hidden states
            last_hidden_state = outputs.last_hidden_state

            # Pass through linear layer to get logits
            logits = mini_lm_linear(last_hidden_state)

            # Flatten the logits and input_ids for cross-entropy loss
            logits_flat = logits.view(-1, logits.size(-1))
            input_ids_flat = input_ids.view(-1)

            # Calculate cross-entropy loss
            loss = F.cross_entropy(logits_flat, input_ids_flat)

            # Backward pass and optimization step
            loss.backward()
            mini_lm_optimizer.step()

    # Save the trained mini-LM
    mini_lm_model.save_pretrained(f"./mini_lm_split_{split_id}")
    torch.save(mini_lm_linear.state_dict(), f"./mini_lm_split_linear_{split_id}")
    tokenizer.save_pretrained(f"./mini_lm_split_{split_id}")

    # Append the trained mini-LM to the list
    mini_lm_models.append((mini_lm_model, mini_lm_linear))


Dataset length for split 0: 1


Dataset length for split 1: 1


Dataset length for split 2: 1


Dataset length for split 3: 1


Dataset length for split 4: 1


Dataset length for split 5: 1


Dataset length for split 6: 0
Skipping training for split 6 as the dataset is empty.
Dataset length for split 7: 1


Dataset length for split 8: 1


Dataset length for split 9: 0
Skipping training for split 9 as the dataset is empty.


In [ ]:
from annoy import AnnoyIndex
import os

# Directory to store the Annoy index files
index_dir = "annoy_index"
os.makedirs(index_dir, exist_ok=True)

# Loop over splits
for split_id, (mini_lm_model, mini_lm_linear) in enumerate(mini_lm_models):
    # Use mini-LM to obtain embeddings for the corresponding split
    split_embeddings = []

    # Create a new dataset for obtaining embeddings
    split_texts = train_df[train_df['Split_ID'] == split_id]['Urdu Story'].tolist()

    if len(split_texts) > 0:
        split_dataset = UrduDataset(split_texts, [0] * len(split_texts), tokenizer)
        split_loader = DataLoader(split_dataset, batch_size=8, shuffle=False)

        with torch.no_grad():
            mini_lm_model.eval()
            mini_lm_linear.eval()
            for batch in tqdm(split_loader, desc=f'Embedding Split {split_id}', leave=False):
                input_ids, attention_mask, _ = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)

                # Forward pass through mini-LM model
                outputs = mini_lm_model(input_ids, attention_mask=attention_mask)

                # Extract last hidden states
                last_hidden_state = outputs.last_hidden_state

                # Pass through linear layer to get logits
                logits = mini_lm_linear(last_hidden_state)

                # Use mean pooling to obtain embeddings
                embeddings = last_hidden_state.mean(dim=1)  # You can modify this based on your requirements
                split_embeddings.append(embeddings.cpu().numpy())

        # Check if split_embeddings is not empty before concatenating and storing
        if split_embeddings:
            # Save the split embeddings along with relevant information
            split_embeddings = np.concatenate(split_embeddings, axis=0)

            # Dimensionality of the embeddings
            embedding_dim = split_embeddings.shape[1]

            # Create an Annoy index with cosine similarity (metric=2)
            annoy_index = AnnoyIndex(embedding_dim, 'angular')

            # Add embeddings to the Annoy index
            for i, embedding in enumerate(split_embeddings):
                annoy_index.add_item(i, embedding)

            # Build the Annoy index
            annoy_index.build(10)  # Adjust the number of trees based on your requirements

            # Save the Annoy index to disk
            annoy_index.save(f"{index_dir}/annoy_index_split_{split_id}.ann")

            # Store any additional information (e.g., split ID, story ID) in a separate file or database

            print(f"Annoy index for split {split_id} created and saved.")
        else:
            print(f"No samples found for split {split_id}. Skipping.")
    else:
        print(f"No samples found for split {split_id}. Skipping.")


In [ ]:
#Load Mini-LM and Annoy Index for the Corresponding Split
from transformers import DistilBertModel, DistilBertTokenizer
from annoy import AnnoyIndex
import torch
import numpy as np

# Load mini-LM model
mini_lm_model = DistilBertModel.from_pretrained('mini_lm_split_7')
mini_lm_model.to(device)

# Load mini-LM linear layer
mini_lm_linear = torch.nn.Linear(mini_lm_model.config.hidden_size, tokenizer.vocab_size).to(device)
mini_lm_linear.load_state_dict(torch.load('mini_lm_split_linear_7'))
mini_lm_linear.eval()

# Load Annoy index
annoy_index = AnnoyIndex(embedding_dim, 'angular')
annoy_index.load('/content/annoy_index/annoy_index_split_7.ann')

# Check the shape of the mini-LM model
#print(f"Mini-LM Model Shape: {mini_lm_model}")
#print(f"Mini-LM Linear Layer Shape: {mini_lm_linear}")


In [ ]:
def get_query_embeddings(query, mini_lm_model, mini_lm_linear, tokenizer, device):
    with torch.no_grad():
        mini_lm_model.eval()
        mini_lm_linear.eval()

        # Tokenize and encode the query
        encoding = tokenizer(query, return_tensors='pt', padding=True, truncation=True)
        input_ids, attention_mask = encoding['input_ids'].to(device), encoding['attention_mask'].to(device)

        # Forward pass through mini-LM model
        outputs = mini_lm_model(input_ids, attention_mask=attention_mask)

        # Extract last hidden states
        last_hidden_state = outputs.last_hidden_state

        # Pass through linear layer to get logits
        logits = mini_lm_linear(last_hidden_state)

        # Use mean pooling to obtain embeddings
        query_embeddings = last_hidden_state.mean(dim=1).cpu().numpy()

    print(f"Shape of query_embeddings: {query_embeddings.shape}")  # Add this line for debugging
    return query_embeddings


In [ ]:
def query_vector_database(query_embeddings, annoy_index, top_n=5):
    # Flatten the query_embeddings to make it 1D
    query_embeddings_flat = query_embeddings.flatten()

    # Get similar splits and distances
    similar_splits, distances = annoy_index.get_nns_by_vector(query_embeddings_flat, top_n, include_distances=True)

    # Return as a list of tuples
    return list(zip(similar_splits, distances))


In [ ]:
# Example usage
user_query = "علی کون تھا؟"
query_embeddings = get_query_embeddings(user_query, mini_lm_model, mini_lm_linear, tokenizer, device)
similar_splits = query_vector_database(query_embeddings, annoy_index, top_n=5)

for split_index, distance in similar_splits:
    urdu_text_of_similar_split = valid_df.iloc[split_index]['Urdu Story']
    print(f"Similar Split Index: {split_index}, Distance: {distance}, Urdu Text: {urdu_text_of_similar_split}")


In [ ]:
#Evaluating Model
from tqdm import tqdm

def evaluate_model_and_annoy(valid_dataset, model, mini_lm_model, mini_lm_linear, tokenizer, annoy_index, top_n=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    correct_predictions = 0
    total_queries = len(valid_dataset)

    for idx in tqdm(range(total_queries), desc="Evaluating Model and Annoy Index"):
        query = valid_dataset['Urdu Story'].iloc[idx]
        true_label = valid_dataset['Split_ID'].iloc[idx]

        # Get query embeddings using the fine-tuned DistilBERT model
        query_embeddings = get_query_embeddings(query, mini_lm_model, mini_lm_linear, tokenizer, device)

        # Get similar splits using the Annoy Index
        similar_splits = query_vector_database(query_embeddings, annoy_index, top_n=top_n)

        # Check if the true label is among the top similar splits
        if true_label in [split_index for split_index, _ in similar_splits]:
            correct_predictions += 1
        # Inside the loop of evaluate_model_and_annoy function
        print(f"True Label: {true_label}")
        print(f"Top Similar Splits: {similar_splits}")


    accuracy = correct_predictions / total_queries
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Correct Predictions: {correct_predictions}/{total_queries}")

# Load the fine-tuned DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained('./distilbert-finetuned').to(device)

# Load mini-LM model
mini_lm_model = DistilBertModel.from_pretrained('mini_lm_split_0').to(device)

# Load mini-LM linear layer
mini_lm_linear = torch.nn.Linear(mini_lm_model.config.hidden_size, tokenizer.vocab_size).to(device)
mini_lm_linear.load_state_dict(torch.load('./mini_lm_split_linear_3'))
mini_lm_linear.eval()

# Load the Annoy Index
annoy_index = AnnoyIndex(embedding_dim, 'angular')
annoy_index.load('./annoy_index/annoy_index_split_5.ann')  # Change the split index based on your saved Annoy Index

# Evaluate the model and Annoy Index
evaluate_model_and_annoy(valid_df, model, mini_lm_model, mini_lm_linear, tokenizer, annoy_index, top_n=5)
